# Sentimen Analisis untuk Pemilu (Pilpres) untuk Memilih Paslon Mana

In [1]:
import pandas as pd

dataFile = pd.read_csv("Project3-news-data.csv", encoding="utf-8")

print('Rows :', dataFile.shape[0])
print('Columns :', dataFile.shape[1])
print('Unique Columns :', dataFile.columns.tolist())
print('Missing Values :', dataFile.isnull().sum().values.sum())

Rows : 19813
Columns : 7
Unique Columns : ['title', 'author', 'publish_date', 'article_text', 'url', 'main_image', 'tag']
Missing Values : 1880


In [2]:
dataFile.dropna(inplace=True)
dataFile.isnull().sum()

title           0
author          0
publish_date    0
article_text    0
url             0
main_image      0
tag             0
dtype: int64

In [3]:
dataFile.head()

,title,author,publish_date,article_text,url,main_image,tag
0,Hasto Ingin Debat Pilpres Pertarungan Gagasan ...,detikNews,2023-12-10 23:31:00+07:00,Sekjen PDIP sekaligus Sekretaris Tim Pemenang ...,https://news.detik.com/pemilu/d-7082492/hasto-...,https://awsimages.detik.net.id/community/media...,"['hasto kristiyanto', 'hasto', 'sekjen pdip', ..."
2,Contoh Format dan Isi Surat Pernyataan Anggota...,detikNews,2023-12-10 22:38:00+07:00,Surat pernyataan KPPS Pemilu 2024 merupakan ba...,https://news.detik.com/pemilu/d-7082424/contoh...,https://awsimages.detik.net.id/community/media...,"['surat pernyataan kpps pemilu 2024', 'pendaft..."
3,9 Ribu Personel Gabungan Dikerahkan Amankan Na...,detikSumut,2023-12-10 22:23:00+07:00,Sekitar 9.000 personel dikerahkan untuk mengam...,https://www.detik.com/sumut/berita/d-7082448/9...,https://awsimages.detik.net.id/community/media...,"['natal', 'tahun baru', 'nataru', 'pengamanan ..."
4,Amarah Ukraina Usai Rusia Bakal Gelar Pilpres ...,detikNews,2023-12-10 22:12:00+07:00,Ukraina marah usai Rusia menjadikan empat wila...,https://news.detik.com/internasional/d-7082454...,https://awsimages.detik.net.id/community/media...,"['ukraina', 'rusia', 'pemilu rusia', 'round-up']"
5,TKN Prabowo-Gibran Buka Pintu untuk Eks KSAD D...,detikNews,2023-12-10 21:59:00+07:00,Sekretaris Tim Kampanye Nasional (TKN) Prabowo...,https://news.detik.com/pemilu/d-7082445/tkn-pr...,https://awsimages.detik.net.id/community/media...,"['tkn', 'prabowo-gibran', 'dudung abdurachman']"


## Pengemlompokan dengan TAg

In [4]:
dataFile['tag'] = dataFile['tag'].apply(lambda x: eval(x))

In [5]:
dataFileExploded = dataFile.explode('tag')

In [6]:
dataFileGroup = dataFileExploded.groupby('tag')[['title','article_text']].agg(list)
dataFileGroup.head()
dataFileGroup = dataFileGroup.reset_index()

In [7]:
!pip install textBlob

In [8]:
from textblob import TextBlob

def calculateSentiment(title, context):
  allText = ' '.join(title) + ' ' + ' '.join(context)
  analysis = TextBlob(allText)
  # Text Blob menghitung sentimen analisisnya
  return analysis.sentiment.polarity

def getTextByTag(data, tag):
  return dataFileGroup[dataFileGroup['tag'].apply(lambda x: tag in x)]

result_sentiment = {}

tags = ['anies baswedan', 'prabowo subianto', 'ganjar pranowo']

for tag in tags:
  newsTag = getTextByTag(dataFileGroup, tag)
  newsTag['sentiment_score'] = newsTag.apply(lambda x: calculateSentiment(x['title'], x['article_text']), axis=1)
  newsTagCompound = newsTag['sentiment_score'].mean()

  result_sentiment[tag] = newsTagCompound

result_sentiment

best_sentiment = max(result_sentiment, key=result_sentiment.get)

C:\Users\jmd\AppData\Local\Temp\ipykernel_14708\3384419118.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newsTag['sentiment_score'] = newsTag.apply(lambda x: calculateSentiment(x['title'], x['article_text']), axis=1)
C:\Users\jmd\AppData\Local\Temp\ipykernel_14708\3384419118.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newsTag['sentiment_score'] = newsTag.apply(lambda x: calculateSentiment(x['title'], x['article_text']), axis=1)
C:\Users\jmd\AppData\Local\Temp\ipykernel_14708\3384419118.py:18

In [9]:
result_sentiment

{'anies baswedan': 0.03324814875728881,
 'prabowo subianto': -0.0006860469524301141,
 'ganjar pranowo': 0.034375715798398286}

In [10]:
print("Sentiment analisis terbaik adalah : ", best_sentiment)

Sentiment analisis terbaik adalah :  ganjar pranowo
